<a href="https://colab.research.google.com/github/dinoelT/YoutubeCode/blob/main/Vgg16_Torch_KernelVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualization of the patterns the kernels of the neural network are looking for. 
Youtube channel: [Leonid's Dreams](https://www.youtube.com/channel/UCvOlcEWLNSm_JbMJVDhcSyw)

In [2]:
import torch
import torchvision.models as models
import numpy as np
from torch import nn, optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from IPython.display import clear_output, display

In [3]:
#Pick a layer to visualize, from conv_1 ... conv_13
layerName = "conv_10"

''' The image will start from the <initialSize>, and will be multiplied with the 
    <scale_factor> for <nrOfScalings> times. In between the scalings, the image
    will be optimized to minimize the loss.
    Example: if imageSize is (100,150) and the nrOfScalings=10 with a
    scaling_factor = 1.2, the final image size will be 
    (100 * 1.2^10, 150 * 1.2^10) = (619, 928)
'''

#Pick the initial image size, that will be resized 
imageSize = (100, 100)

#Pick the number of times the image will be scaled
nrOfScalings = 10

#Choose the scaling factor
scale_factor = 1.2

#Pick the number of filters you want to visualize
NrOfFiltersToVisualize = 10

In [4]:
def getVgg16Model():
  """ Returns a Vgg16 model with AveragePool instead of Maxpool 
      and with naming convention: avgpool_x, conv_x, relu_x
  """
  initialModel = models.vgg16(pretrained=True)

  modifiedModel = torch.nn.modules.container.Sequential()
  #Dictionary with the index of the layers for naming
  index_dict = {'avgpool': 1, 'conv':1, 'relu':1}

  for layer in initialModel.features:
    
    if isinstance(layer, torch.nn.MaxPool2d):
      name = 'avgpool'
      i = index_dict[name]
      modifiedModel.add_module(name+'_'+str(i), torch.nn.AvgPool2d((2,2)))
      index_dict[name] = index_dict[name] +  1
    elif isinstance(layer, torch.nn.Conv2d):
      name = 'conv'
      i = index_dict[name]
      modifiedModel.add_module(name+'_'+str(i),layer)
      index_dict[name] = index_dict[name] +  1
    elif isinstance(layer, torch.nn.ReLU):
      name = 'relu'
      i = index_dict[name]
      modifiedModel.add_module(name+'_'+str(i),layer)
      index_dict[name] = index_dict[name] +  1 
    else:
      break
  return modifiedModel


def getRandomImage(imageSize):
  """ Returns a random image with shape (1,3,y,x)
      imageSize = tuple([y,x])
  """
  img = np.random.uniform(0, 1,(1,*imageSize, 3))
  # Normalizing input for vgg16
  mean = [0.485, 0.456, 0.406]
  std = [0.229, 0.224, 0.225]
  img = (img - mean) / std
  img = np.transpose(img, axes = (0,3,1,2)).astype("float32")
  generatedImage = torch.tensor(img.copy(), requires_grad=True)
  return generatedImage
 

def Loss_lpf(featureMap,indexFilterMaximize, generatedImage, alpha):
  """ Calculates the loss with low pass filter"""
  a = torch.sum(featureMap[:,indexFilterMaximize,:,:])
  b = lowPassFilterLoss(generatedImage)
  loss = b/(a + 0.1)
  return loss

def Loss(featureMap,indexFilterMaximize, alpha):
  """ Calculates the loss"""
  a = torch.sum(featureMap[:,indexFilterMaximize,:,:])
  loss = alpha/(a + 0.1)
  return loss

def tensorToImage(tensor):
  """ Transforms image tensor into numpy array with shape (y,x,3)"""
  image = tensor.cpu().detach().numpy()[0]
  image = np.transpose(image, axes = (1,2,0))
  return image

def lowPassFilterLoss(inputImg):
  """ Calculates the low pass filter loss on the horizontal and vertical axes.
  inputImg size : (1, 3, y, x)
  """

  verticalFilter = torch.sum( torch.abs(inputImg[:,:, :-1, :] - inputImg[:,:, 1:, :] ))
  horizontalFilter = torch.sum( torch.abs(inputImg[:,:,:, :-1] - inputImg[:,:,:, 1:] ))

  return verticalFilter + horizontalFilter

def getMaxFiltersIndexes(imageSize, nrFilters):
  """ Returns the indexes of the kernels that had higher activations when applied
  to a random image(if the activations are too low or zero, the gradient is zero)
  imageSize = tuple([y,x])
  nrFilters - number of filter indexes to be returned, sorted by the value of 
  the activations (from high to low)
  """
  inputImage = getRandomImage(imageSize).to(device)

  out = model(inputImage)

  a = features.cpu().detach().numpy()[0]
  s = np.sum(a, axis = (1,2))
  s_sorted =  np.sort(s)[-nrFilters:]
  min_element = np.floor( np.amin(s_sorted))
  return np.where(s > min_element)[0]


In [ ]:
#handle - holds a reference to the hook we added, in case we want to remove it
handle = None

#features - this is where the output of the layer will be stored
features = None

def featureSaveHook(module, input, output):
  global features
  features = output

def addHook(model, layerName):
  global handle

  #Clear all hooks
  if handle is not None:
    handle.remove()

  layerIndex = list(dict(model.named_children()).keys()).index(layerName)
  handle = model[layerIndex].register_forward_hook(featureSaveHook)


#Create model
model = getVgg16Model()

#Create hook for saving features
addHook(model, layerName)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Used device:", device)
model = model.to(device)

layerList = [layer for layer in dict(model.named_children()).keys()]
print("Layer names:",layerList)

#create a folder to save the images in 
import os

path = os.getcwd()
os.mkdir(layerName)  

In [6]:
filterIndexes = getMaxFiltersIndexes(imageSize, NrOfFiltersToVisualize)
print("Filter indexes:",filterIndexes)

for filterIndex in filterIndexes:
  inputImage = getRandomImage(imageSize)
  #print(inputImage.shape)
  inputImage = inputImage.to(device).detach().requires_grad_(True)

  for i in range(nrOfScalings):
    inputImage = F.interpolate(inputImage, scale_factor = scale_factor)
    inputImage = inputImage.to(device).detach().requires_grad_(True)
    optimizer = optim.Adam([inputImage], lr=0.1)
    print("\n<<<<<<<<Iteration",i,">>>>>>>>>")
    print("Image shape:", inputImage.shape,"\n")
    for epoch in range(30):
      featureMaps = model(inputImage)
      loss = Loss_lpf(features, filterIndex, inputImage, 10e7)
      #loss = Loss(features, filterIndex, 1000)
      print(loss.item())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      with torch.no_grad():
        inputImage[:] = inputImage.clamp(0, 1)
  image = tensorToImage(inputImage)
  plt.imsave(layerName+"/"+layerName + "_filter"+str(filterIndex)+"_lpf.png", image)
  clear_output()


In [ ]:
#Save folder as an archive
import shutil
shutil.make_archive(layerName, 'zip', layerName)